In [180]:
import numpy as np
import pandas as pd

In [181]:
dataset = pd.read_csv("iris.data.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)

In [182]:
X = np.array(dataset)[:,:-1]
print(X[:5])
print(X.shape)

[[5.5 2.6 4.4 1.2]
 [6.7 3.0 5.2 2.3]
 [6.8 3.2 5.9 2.3]
 [4.6 3.4 1.4 0.3]
 [6.1 2.8 4.0 1.3]]
(150, 4)


In [183]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)


In [184]:
Y = dataset.Class_name
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
#basically converts output 2 into 0 0 1 (2nd one is true and others are false)
Y[:5]

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [185]:
#splitting test, train and validation data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [186]:
def InitializeWeights(nodes):
    layers, weights = len(nodes), []
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
                for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights

In [187]:
def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    activations = np.array(x, dtype=float)
    for j in range(layers):
        activation = Sigmoid(np.dot(layer_input, weights[j].T))
        np.append(activation, activations)
        layer_input = np.append(1, activation) # Augment with bias

    return activations

In [188]:
def BackPropagation(y, lr, activations, weights, layers):

    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal)

    for j in range(layers, 0, -1):
        currActivation = activations[j]

        if(j > 1):
            prevActivation = np.append(1, activations[j-1])
        else:
            prevActivation = activations[0]

        delta = np.multiply(error, SigmoidDerivative(currActivation))
        weights[j-1] = weights[j-1] + lr * np.multiply(delta.T, prevActivation)

        w = np.delete(weights[j-1], [0], axis=1)
        error = np.dot(delta, w)
        
    return weights

In [189]:
def Train(X, Y, lr, weights):
    layers = len(weights)

    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augment feature vector
        activations = ForwardPropagation(x, weights, layers)
        weights = BackPropagation(y, lr, activations, weights, layers)
        
    return weights

In [190]:
def Sigmoid(x):
    ar = np.array(x, dtype=float)
    return 1/(1 + np.exp(-ar))

def SigmoidDerivative(x):
    return np.multiply(x,1-x)

In [191]:
def FindMaxActivation(output):
    m, index = output[0], 0
    m=np.array(m, dtype=float)
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    return index

In [192]:
def Predict(item, weights):
    layers = len(weights)
    item = np.append(1, item)

    activations = ForwardPropagation(item, weights, layers)
    activations = np.matrix(activations)
    outputFinal = activations[-1].A1
    
    index = FindMaxActivation(outputFinal)
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1
    return y

In [193]:
def Accuracy(X, Y, weights, display=False):
    correct = 0
    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = Predict(x, weights)
        if display == True:
            print("\n\nInput:\n",x,"\nPredicted:\n",guess,"\nActual:\n",y)
        if(y == guess):
            correct += 1
        elif display == True:
            print("mispredicted")
    return correct/len(X)

In [194]:
def NeuralNetwork(X_train, Y_train, X_val=None, Y_val=None, epochs=10, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)
    
    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)
        if(epoch % 20 == 0):
            print("Epoch {}".format(epoch))
            print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
            if X_val.any():
                print("Validation Accuracy:{}".format(Accuracy(X_val, Y_val, weights)))
    return weights

In [195]:
f = len(X[0])
o = len(Y[0])
layers = [f, 6, 9, o]
lr, epochs = 0.15, 100
weights = NeuralNetwork(X_train, Y_train, X_val, Y_val, epochs=epochs, nodes=layers, lr=lr)
print("Final weights:\n",weights)
print("Testing Accuracy: {}".format(Accuracy(X_test, Y_test, weights, display = True)))

ValueError: operands could not be broadcast together with shapes (3,) (5,) 